In [3]:
from bs4 import BeautifulSoup
import cv2
import os
import numpy as np
import random
path= "croped/"
output= "croped_colored/"

In [4]:
def delete_small(dic):
    """Delete all textregions which are too small. They are probably errors from the segmentation step

    Parameters
    ----------
    dic:  dicitonary
        key:    is the name of the textregion (in the format: "region_00xx")
        values: Its a list. List[0] contains all coordinates which the segmentor recognized as textarea. List[1] is the reading order
 
    Returns
    -------
    new_dic: dictionary
        Is the same format like the param "dic", but now has a few entrys less.  
    """
    
    dic = dict(sorted(dic.items(), key=lambda item: int(item[1][1]))) #sort by reading order
    new_dic = {}

    count = 0 
    for k,v in dic.items():
        start,end = get_start_end(v[0])
        
        area = (end[0]-start[0])*(end[1]-start[1]) #calculate area of rectangle
        
        #delete regions which have an area smaller then a certain area
        if area <= 1500:
            count +=1
            continue
        else:
            new_dic[k] = [v[0],int(v[1])-count]

    return new_dic
            

In [5]:
## das auch nochmal überarbeiten (Vor allem case wenn bild in Spalte mit drinnen ist)

def check_if_near(top_left, next_top_left, bottom_right, next_bottom_right,height_dist):
    """Check if two boundingboxes (above each other) with certain x and y values are near to each other

    Parameters
    ----------
    bottom_left: int
        x-coordinate of the top left of the upper bounding box
    next_bottom_left: int
        x-coordinate of the top left of the lower bounding box
    bottom_right: int
        x-coordinate of the bottom right of the upper bounding box
    next_bottom_right: int
        x-coordinate of the bottom right of the lower bounding box
    height_dist: int
        distance between the y-coordinate of the bottom right_corner of the upper bounding box and the y-coordinate of the top left corner of the lower boundingbox
        
    Returns
    -------
    True:
        If two boundingboxes are in the same column and have nearly the same x values on the left and right side and the distance between is very small
    False:
        If not so.
    """
    
    
    if (abs(top_left-next_top_left) + abs(bottom_right-next_bottom_right) <40) and height_dist <20:
     
        return True
    else:
        return False

In [6]:
def combine_regions_in_column(dic):
    
    """This function combines textregions (boundingboxes) in one column, which have nearly the same x values on the left and right side, and a small y distance between two regions.

    Parameters
    ----------
    dic:  dicitonary
        key:    is the name of the textregion (in the format: "region_00xx")
        values: Its a list. List[0] contains all coordinates which the segmentor recognized as textarea. List[1] is the reading order
 
    Returns
    -------
    new_dic: 
        key:    New Ids for the new reading order
        values  A list of strings, which have the name of the textregions, which are now combined in one bigger region. 
    """
    
    dic = dict(sorted(dic.items(), key=lambda item: int(item[1][1]))) #sort by reading order
    new_dic = {}
   
    count = 0
    
    for i,(k,v) in enumerate(list(dic.items())):
       
    
        #two dummyelements for very first and the very last textregion on the page
        if i==0:
            last_entry = [0,["0,0",0]]
        else:
            last_entry = list(dic.items())[i-1]
        
        if i==len(dic.items())-1:
            next_entry = [0,["0,0",0]]
        else:
            next_entry = list(dic.items())[i+1]
        
        
        #here we calculate the top left and bottom right points for the last, the present and the next bounding bos
        start,end = get_start_end(v[0])
        n_start, n_end = get_start_end(next_entry[1][0])
        l_start, l_end = get_start_end(last_entry[1][0])
        
        
       
        if check_if_near(start[0],n_start[0], end[0], n_end[0], abs(end[1]-n_start[1])) and (int(v[1])+1 == next_entry[1][1]):
           
            if count not in new_dic:
                
                new_dic[count]=[k]
            else: 
                new_dic[count].append(k)
        
        elif check_if_near(l_start[0],start[0], l_end[0], end[0], abs(l_end[1]-start[1])) and (int(v[1])-1 == last_entry[1][1]):
            if count not in new_dic:
                
                new_dic[count]=[k]
            else: 
                new_dic[count].append(k)
            
            count+=1
        else:
            count+=1
            new_dic[count]=[k]
            count+=1
     
    #rename the keys of the dictionary for a new clean order
    new_dic = {index: value for index, (_, value) in enumerate(new_dic.items())}
    
    #calculate possible header in the same column
    new_dic = possible_headline_in_column(dic, new_dic)
    
    new_dic = {index: value for index, (_, value) in enumerate(new_dic.items())}
    
    return new_dic
    
        
    

In [7]:
combine_regions_in_column(regs)

NameError: name 'regs' is not defined

In [8]:

def possible_headline_in_column(dic, combined):
    
    """This function checks if one or two textregions above other textregions could be a title and a subtitle, but only for one column headlines!

    Parameters
    ----------
    dic:  dicitonary
        key:    is the name of the textregion (in the format: "region_00xx")
        values: Its a list. List[0] contains all coordinates which the segmentor recognized as textarea. List[1] is the reading order
    combined: dictionary
        key:    is just a new id for the textregions, because there are now combined, and the bigger combined regions gets a new id
        values: Its a list. The list contains the regions of already combined regions (from heuristics before)
    
    Returns
    -------
    new: 
        a dictionary quite similar to the dictionary "combined". But now possible headers and there matching textareas are combined in one entry in the dictionary
    """

    new = {k:v for k,v in combined.items()}
    
    
    for i,(k,v) in enumerate(list(combined.items())):
        if i==0:
            continue
        
        last_entry = combined[i-1]
        
        # in this section we try to combine Title and subtitle. But also Title and text if there is no subtitle
        if len(last_entry) == 1:
            start_sec,end_sec= get_start_end(dic[v[0]][0])
            start_first_headline, end_first_headline = get_start_end(dic[last_entry[0]][0])
            
            #here you can change the parameters for the headline bounding boxes
            if (start_sec[0]-200 <= start_first_headline[0] <= end_sec[0]) and (start_sec[0] <= end_first_headline[0] <= end_sec[0]+80) and (abs(start_first_headline[1]-start_sec[1])<50):
                    new[k].insert(0,last_entry[0])
                    new[k-1].remove(last_entry[0])
        
        
        #this sections trys to combine a recognized title and subtitle region with the matching textregion
        if len(last_entry) == 2:
            start_sec,end_sec= get_start_end(dic[v[0]][0])
            start_first_headline, end_first_headline = get_start_end(dic[last_entry[0]][0])

            if (start_sec[0]-40 <= start_first_headline[0] <= end_sec[0]) and (start_sec[0] <= end_first_headline[0] <= end_sec[0]+40) and (abs(start_first_headline[1]-start_sec[1])<100):
                    new[k].insert(0,last_entry[0])
                    new[k].insert(1,last_entry[1])
                    new[k-1].remove(last_entry[0])
                    new[k-1].remove(last_entry[0])

    new={k: v for k, v in new.items() if len(v)>0} 
    return new
    


In [9]:


def get_start_end(coord_list):
    
    """Gets the top left corner and the right bottom corner of each bounding box"

    Parameters
    ----------
    coord_list : str
        Its a list of coordinates from a xml file for the region of text (polygonomial)

    Returns
    -------
    start:
        a tuple of the smallest x and y value of the region (top left corner)
    end:
        a tuple of the maximum x and y value of the region (bottom right corner)
    """
    
    coord_list = coord_list.split(" ")
    coord_list = [tuple(map(int,tuple(x.split(",")))) for  x in coord_list]
    x_max = max(coord_list,key=lambda x:x[0])
    x_min = min(coord_list,key=lambda x:x[0])
    y_max = max(coord_list,key=lambda x:x[1])
    y_min = min(coord_list,key=lambda x:x[1])
        
        
    ## Coordinate System starts in the top left corner of the picture!
    start = (x_min[0],y_min[1])
    end = (x_max[0], y_max[1])
    
    return start,end


In [16]:
count= 1
for filename in os.listdir(path):
    if not filename.endswith('.xml'): continue
    fullname = os.path.join(path, filename)

    with open(fullname, 'r') as f:
        data = f.read()
        data= BeautifulSoup(data, "xml")
        
        regions = data.find_all("TextRegion")
        
        
        separator_list = data.find_all("SeparatorRegion")

        order = {x["regionRef"]:x["index"]for x in data.find_all("RegionRefIndexed")} #this is already the sorted order beginning from order[0]
   
    number = fullname.split("/")[-1].split(".")[0]

    print(fullname)
    textlines={}
    
    for reg in regions:
        #[reg.get("id")]
        for line in reg.find_all("TextLine"):
            
            if reg.get("id") not in textlines:
                textlines[reg.get("id")] = [[line.get("id"),line.find("Coords")["points"]]]
                
            else: 
                textlines[reg.get("id")].append([line.get("id"),line.find("Coords")["points"]])
                
    regs={}
    for reg in regions:
        regs[reg.get("id")] = [reg.find("Coords")["points"],order[reg.get("id")]]
    
    #delete all too small wrong annotated regions
    regs = delete_small(regs)
    
    #combined regions in one column
    combined = combine_regions_in_column(regs)
    
    separators=[]
    for separ in separator_list:
        separators.append(separ.find("Coords")["points"])
        
    img = cv2.imread(path + number + ".jpg")
    overlay = img.copy()
    
    for sep in separators:

        
        start,end = get_start_end(sep)
       # col = tuple(random.randrange(256) for _ in range(3))
        #print(col)
        img = cv2.rectangle(img, start, end, (0,0,255), -1)

    
    
    """
    for k,v in regs.items():
        
        #v[0] is the coordinate list, v[1] is the reading order rank
        start,end = get_start_end(v[0])


        #assigns a random colour to each rectangle
        col = tuple(random.randrange(256) for _ in range(3))
        img = cv2.rectangle(img, start, end, col, -1)

        #puts the reading order to the rectangles
        
        #img = cv2.putText(img, str(v[1]), (start[0]+5,start[1]+20),  cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
   
    """
    
    for k,v in combined.items():
        
        
        col = tuple(random.randrange(256) for _ in range(3))
        textposition = (get_start_end(regs[v[0]][0]))        
        
        for reg in v:
        
            #v[0] is the coordinate list, v[1] is the reading order rank
            start,end = get_start_end(regs[reg][0])

            #assigns a random colour to each rectangle
          
            img = cv2.rectangle(img, start, end, col, -1)
            
            if reg not in textlines:
                continue
            else:
                for line in textlines[reg]:
                    s,e = get_start_end(line[1])
                    img = cv2.rectangle(img, s, e, (0,0,255), 1)


            #puts the reading order to the rectangles
        
        img = cv2.putText(img, str(k), (textposition[0][0]+5, textposition[0][1]+25),  cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 4)
    
    
    alpha =0.4
    img = cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0)

    result = output + "result_" +number +".jpg"
    print(result)
    cv2.imwrite(result, img)
    count +=1
    

croped/croped_00000009.xml
croped_colored/result_croped_00000009.jpg
croped/croped_00000010.xml
croped_colored/result_croped_00000010.jpg
croped/croped_00000015.xml
croped_colored/result_croped_00000015.jpg
croped/croped_00000017.xml
croped_colored/result_croped_00000017.jpg
croped/croped_00000018.xml
croped_colored/result_croped_00000018.jpg
croped/croped_00000005.xml
croped_colored/result_croped_00000005.jpg
croped/croped_00000014.xml
croped_colored/result_croped_00000014.jpg
croped/croped_00000016.xml
croped_colored/result_croped_00000016.jpg
croped/croped_00000001.xml
croped_colored/result_croped_00000001.jpg
croped/croped_00000002.xml
croped_colored/result_croped_00000002.jpg
croped/croped_00000012.xml
croped_colored/result_croped_00000012.jpg
croped/croped_00000004.xml
croped_colored/result_croped_00000004.jpg
croped/croped_00000008.xml
croped_colored/result_croped_00000008.jpg
croped/croped_00000006.xml
croped_colored/result_croped_00000006.jpg


## nächsten Schritte:

-mehrspaltige Überschriften erkennen --> Textlinehöhe ist größer als im durchschnittlichen Text
-Artikel unter mehrspaltigen Überschriften zusammenbekommen
-spaltenende Artikel in nächster Spalte weiter?
